In [1]:
from transliterate.phonemizer import ThaiPhonemizer
import os
import sys
from tqdm import tqdm
import re
import pandas as pd
from pythainlp.util import maiyamok

sys.path.append('..')

from utils.text_util import clean_text_cv

In [2]:
phonemizer = ThaiPhonemizer()

commonvoice_sentence_path = '../data/converted/commonvoice-to-gigaspeech/txt'
log_path = './error_log.txt'

In [ ]:
errors = []
for speaker in os.listdir(commonvoice_sentence_path):
    speaker_path = os.path.join(commonvoice_sentence_path, speaker)
    
    file_list = os.listdir(speaker_path)
    progress_bar = tqdm(file_list, desc=f"Processing {speaker}", unit="file")

    for file in progress_bar:
        file_path = os.path.join(speaker_path, file)
        with open(file_path, 'r') as f:
            line = f.readline()
        try:
            phonemizer.phonemize(line)
        except:
            errors.append(line)
        
        progress_bar.set_postfix(total_errors=len(errors))

with open(log_path, 'w', encoding='utf-8') as f:
    f.writelines([error + '\n' for error in errors])


Processing cv134: 100%|██████████| 26698/26698 [1:44:54<00:00,  4.24file/s, total_errors=598]  


In [4]:
processed_log_path = './processed_error_log.txt'

with open(log_path, 'r', encoding='utf-8') as f:
    errors = f.readlines()

processed_errors = []
for error in tqdm(errors):
    cleaned_error = clean_text_cv(error)
    try:
        phonemizer.phonemize(cleaned_error)
    except:
        processed_errors.append(cleaned_error)

with open(processed_log_path, 'w', encoding='utf-8') as f:
    f.writelines([error + '\n' for error in processed_errors])

100%|██████████| 598/598 [03:29<00:00,  2.85it/s]


In [5]:
print(f"Total errors: {len(errors)}")
print(f"Total processed errors: {len(processed_errors)}")

Total errors: 598
Total processed errors: 474


# Try Using PyThaiNLP to Handle ๆ

In [ ]:
exceptions = {
    'เธอเป็นคนดีมาก ๆ': 'เธอเป็นคนดีมากมาก',
    'ให้ประสานงานกับสมาชิกคนอื่น ๆ และหารือเกี่ยวกับปัญหานี้ในภายหลัง': 'ให้ประสานงานกับสมาชิกคนอื่นคนอื่น และหารือเกี่ยวกับปัญหานี้ในภายหลัง',
    'ช่างเป็นสวนหลังบ้านที่ดีงามมาก ๆ จริง ๆ คุณนาย !': 'ช่างเป็นสวนหลังบ้านที่ดีงามมากมาก จริงจริง คุณนาย !',
    'ฉันอาจจะผ่านเรื่องเล็ก ๆ น้อย ๆ นั้นไปได้ด้วยดี': 'ฉันอาจจะผ่านเรื่องเล็กเล็ก น้อยน้อย นั้นไปได้ด้วยดี',
    'มันทำให้รู้สึกสดชื่นจริง ๆ ที่ได้นั่งท่ามกลางบรรยากาศที่มีแต่ลมพัดเย็น ๆ ที่แสนสบาย': 'มันทำให้รู้สึกสดชื่นจริงจริง ที่ได้นั่งท่ามกลางบรรยากาศที่มีแต่ลมพัดเย็นเย็น ที่แสนสบาย',
    'ซาลกำลังทำให้คนอื่น ๆ โกรธ': 'ซาลกำลังทำให้คนอื่นอื่น โกรธ',
    'มีทันตแพทย์และแพทย์คนอื่น ๆ ในคลินิกนี้': 'มีทันตแพทย์และแพทย์คนอื่นอื่น ในคลินิกนี้',
    'ใครพูดรัว ๆ ติดกันได้นาน ๆ บอกเราด้วย': 'ใครพูดรัวรัว ติดกันได้นานนาน บอกเราด้วย',
    '“ผมเสียใจจริง ๆ ที่ต้องไปครับท่าน” ฉันตอบกลับ': '“ผมเสียใจจริงจริง ที่ต้องไปครับท่าน” ฉันตอบกลับ',
    'ใจเย็น ๆ ค่ะ ไม่ต้องตื่นเต้น': 'ใจเย็นเย็น ค่ะ ไม่ต้องตื่นเต้น',
    'เรื่องนี้มันดีมาก ๆ': 'เรื่องนี้มันดีมากมาก',
    'คุณผู้หญิงเหมือนเด็กเล็ก ๆ': 'คุณผู้หญิงเหมือนเด็กเล็กเล็ก',
    "'ช่างเป็นดอกไม้ดอกใหญ่ ๆ สมอย่างที่ต้องเป็นจริง ๆ !' เธอแสดงความคิดต่อมาของเธอ": "'ช่างเป็นดอกไม้ดอกใหญ่ใหญ่ สมอย่างที่ต้องเป็นจริงจริง !' เธอแสดงความคิดต่อมาของเธอ",
    'คนอื่น ๆ เดินตามรอยของเขา' :'คนอื่นอื่น เดินตามรอยของเขา'
}

def handle_maiyamok(text: str, exceptions: dict = None) -> str:
    text = text.strip()
    if 'ๆ' not in text:
        return text
    if exceptions:
        if text in exceptions:
            return exceptions[text]
    chunks = maiyamok(text)
    processed_text = ''.join(chunks)
    return processed_text

In [40]:
with open(log_path, 'r', encoding='utf-8') as f:
    errors = f.readlines()

lines = set()
for error in errors:
    if 'ๆ' in error:
        error = error.strip()
        processed_maiyamok = handle_maiyamok(error, exceptions)
        processed_punct = clean_text_cv(processed_maiyamok)
        error = clean_text_cv(error)
        lines.add((error, processed_punct))

# save lines to csv
save_path = './error_handling.txt'
with open(save_path, 'w', encoding='utf-8') as f:
    for line in lines:
        f.write(f"{line[0]}\n{line[1]}\n\n")

In [8]:
replace_dict = {
    "Facebook": "เฟซบุ๊ก",
    "softmax": "ซอฟต์แม็กซ์",
    "Astroturf": "แอสโตรเทิร์ฟ",
    "Burke": "เบิร์ก",
    "whilst": "ไวล์สท์",
    "Kenny": "เคนนี",
    "Flickr": "ฟลิกเกอร์",
    "Asperger": "แอสเพอร์เกอร์",
    "Johanna": "โจแอนนา",
    "C" : "ซี",
    "section": "เซคชัน",
    "Mr Lincoln": "มิสเตอร์ ลินคอล์น",
    "Brexiteers": "เบร็กซิทเทียร์ส",
    "Brexit": "เบร็กซิท"
}

In [9]:
processed_log_path = './maiyamok_handled_error_log.txt'

with open(log_path, 'r', encoding='utf-8') as f:
    errors = f.readlines()

processed_errors = []
exception_count = 0  # Counter for exceptions

with tqdm(total=len(errors), desc="Processing Errors (Exceptions: 0)") as pbar:
    for error in errors:
        maiyamok_handled = handle_maiyamok(error, exceptions)
        cleaned_error = clean_text_cv(maiyamok_handled, replace_dict)
        # cleaned_error = re.sub(r'\s+', '', cleaned_error)
        try:
            phonemizer.phonemize(cleaned_error)
        except:
            processed_errors.append(cleaned_error)
            exception_count += 1  # Increment exception count
            pbar.set_description(f"Processing Errors (Exceptions: {exception_count})")  # Update progress bar text
        pbar.update(1)  # Update progress bar for each error

with open(processed_log_path, 'w', encoding='utf-8') as f:
    f.writelines([error + '\n' for error in processed_errors])


Processing Errors (Exceptions: 1):   0%|          | 0/598 [00:00<?, ?it/s]

Processing Errors (Exceptions: 180): 100%|██████████| 598/598 [01:57<00:00,  5.10it/s]
